In [ ]:
!pip install git+https://github.com/huggingface/transformers@main > /dev/null
!pip install accelerate  > /dev/null
!pip install  simpletransformers==0.63.3  > /dev/null

  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-hdlitq6d


In [ ]:
!nvidia-smi

Mon Nov 21 00:21:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch.utils.data import Dataset, random_split
from simpletransformers.language_modeling import LanguageModelingModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
import gc
import os
import logging
import pandas
import torch
import gc
import pandas
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
import logging
import pandas

In [ ]:
model_name = "fuckswithducks,Poem_for_your_sprog"

In [ ]:
parent_directory = "/content/drive/MyDrive/RawData"

data_dir = f"{parent_directory}/data/"

model_output_dir = f"{parent_directory}/{model_name}"

tokenizer_path = f"{model_output_dir}"

training_data_path = f"{data_dir}/{model_name}-training.csv"

In [ ]:
def has_valid_line(input_line: str) -> bool:
    bad_words_found = []
    black_list = ["[removed]", "[deleted]"]
    for word in black_list:
        if input_line.lower().__contains__(word.lower()):
            print(f":: Line contains word {word}... Skipping")
            bad_words_found.append(input_line)

    return len(bad_words_found) == 0

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
def token_length_appropriate(prompt) -> bool:
    """
    Ensures that the total number of encoded tokens is within acceptable limits.
    :param tokenizer: An instance of the tokenizer being used.
    :param prompt: UTF-8 Text that is assumed to have been processed.
    :return: True if acceptable.
    """
    tokens = tokenizer.tokenize(prompt)
    if len(tokens) > 1024:
        print(f":: Tokens for model input is > {1024}. Skipping input")
        return False
    else:
        return True

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [ ]:
df = pandas.read_csv(training_data_path)

conversations = list(df['TrainingString'])

valid_lines = []

for conversation in conversations:
    if token_length_appropriate(conversation) and has_valid_line(conversation):
        valid_lines.append(conversation)

:: Line contains word [removed]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [removed]... Skipping
:: Tokens for model input is > 1024. Skipping input
:: Line contains word [removed]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Tokens for model input is > 1024. Skipping input
:: Tokens for model input is > 1024. Skipping input
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [deleted]... Skipping
:: Line contains word [removed]... Skipping
:: Tokens for model input is > 1024. Skipping input
:: Line contains word [removed]... Skipping
:: Line contains word [removed]... Skipping
:: Line contains word [removed]... Skipping


In [ ]:
generator = torch.Generator()

generator.manual_seed(0)

print(f":: Total Number Of Samples {len(valid_lines)}")

train_size = int(0.9 * len(valid_lines))

train_dataset_file, eval_dataset_file = random_split(list(valid_lines), [train_size, len(valid_lines) - train_size], generator=generator)

with open("train.txt", 'w', encoding="utf-8") as train_out, open("eval.txt", "w", encoding="utf-8") as eval_out:
        train_out.writelines([repr(line)[1:-1] + "<|endoftext|>" + "\n" for line in train_dataset_file])
        eval_out.writelines([repr(line)[1:-1] + "<|endoftext|>" + "\n" for line in eval_dataset_file])


:: Total Number Of Samples 6457


In [ ]:
gc.collect()

torch.cuda.empty_cache()

In [ ]:
gc.collect()

torch.cuda.empty_cache()
args = {
    "overwrite_output_dir": True,
    "learning_rate": 1e-4,
    "gradient_accumulation_steps": 100,
    "dataset_type": "simple",
    "sliding_window": True,
    "max_seq_length": 1024,
	"mlm": False, # has to be false for gpt-2
    "evaluate_during_training": True,
    "use_cached_eval_features": True,
    "evaluate_during_training_verbose": True,
    "save_optimizer_and_scheduler": False,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": True,
    "save_steps": -1, 
    "train_batch_size":6,
    "num_train_epochs":12,
    "output_dir": f"{model_output_dir}/",
	"best_model_dir": f"{model_output_dir}/best_model"
}
model = LanguageModelingModel("gpt2", "gpt2-large", args=args)
model.train_model(train_file="train.txt", eval_file="eval.txt", args=args, verbose=True)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

  0%|          | 0/5811 [00:00<?, ?it/s]

...

Running Epoch 8 of 12:   0%|          | 0/2084 [00:00<?, ?it/s]

Running Epoch 9 of 12:   0%|          | 0/2084 [00:00<?, ?it/s]

Running Epoch 10 of 12:   0%|          | 0/2084 [00:00<?, ?it/s]

Running Epoch 11 of 12:   0%|          | 0/2084 [00:00<?, ?it/s]

(240,
 {'global_step': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240],
  'perplexity': [tensor(5.0603),
   tensor(4.3418),
   tensor(4.1166),
   tensor(3.9884),
   tensor(3.8965),
   tensor(3.8257),
   tensor(3.7704),
   tensor(3.7330),
   tensor(3.6996),
   tensor(3.6838),
   tensor(3.6731),
   tensor(3.6685)],
  'eval_loss': [1.621417106973364,
   1.4682932995735332,
   1.4150256187357801,
   1.3833986820375666,
   1.3600842811008718,
   1.3417379994341667,
   1.3271684628534823,
   1.3172011923282705,
   1.3082216794503496,
   1.3039467827753817,
   1.30104798195844,
   1.2997723184050398],
  'train_loss': [1.9278703927993774,
   1.7722634077072144,
   1.842545509338379,
   2.242475986480713,
   1.9166102409362793,
   1.4668972492218018,
   0.4580928087234497,
   0.9497575759887695,
   1.3218951225280762,
   1.6902949810028076,
   1.1737172603607178,
   1.954690933227539]})

In [ ]:
import re

def capture_tag(test_string: str, expected_tag: str):
    regex = r"\<\|(.*)\|\>"

    matches = re.finditer(regex, test_string, re.MULTILINE)

    for matchNum, match in enumerate(matches, start=1):
        
        print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
        if match.group() == expected_tag:
            print(f"{match.group()} {expected_tag}")
            return_string = test_string.replace(match.group(), "")
            return return_string

        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            
            print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs

text_model_generator = LanguageGenerationModel("gpt2",  f"/content/drive/MyDrive/RawData/gpt2-models/SportsFanGhost-Bot", args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	})


prompt = "<|soss r/dalle2|><|sot|>Detailed scientific diagram depicting the anatomy of a tomato, full colour, realistic<|sost|>https://i.imgur.com/7adBOXn.jpg<|sor u/AsterJ|>It's going to be sad day when it learns to properly spell.  I feel like this era is a fleeting moment in AI history.  We must cherish it.<|eor|><|sor|>"

body = "The religion of Barryism always enjoyed decent popularity on this sub, but after the destruction of u/Creator_GUTENMAN and fall of GUTENMANism it began to gow astoundingly fast. What is your opinion on the religion of Barryism? Will it become the dominant religion of r/subsimgpt2interactive?"
prompt = f"<|soss r/subsimgpt2interactive|><|sot|>{body}<|eot|><|sost|><|eost|><|sor|>"

import re
regex = r"\<\|(.*)\|\>"


reply = None
refresh_args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	}
while reply is None:
	for text in text_model_generator.generate(prompt=prompt, args=refresh_args, verbose=True):
		foo = text.replace(prompt, "\n")
		result = capture_tag(foo, "<|eor|>")
		if result != None:
			reply = result
			break
print(reply)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-3-2f2ffc8cbd9e>:34 in <module>                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'capture_tag' is not defined

In [ ]:
text_model_generator = LanguageGenerationModel("gpt2",  f"{model_output_dir}/best_model", args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	})
refresh_args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	}

original:str = "<|ososs r/Futurology|><|sot|>This Waifu Does Not Exist | Refresh the page to get a new anime girl, generated by GANs!<|eot|><|sost|>http://www.thiswaifudoesnotexist.net/index.html<|eost|><|sor u/eugd|>I think this is actually the case. They seem to have used some set of ecchi pinups where they're eating (presumably a lot of strawberries and cream and whipped-topping sponge cakes).<|eor|><|soopr|>I'm glad that's the case because [I was starting to get concerned](https://i.imgur.com/rScRZqT.png).<|eoopr|>"

prompt: str = "<|ososs r/foo|><|sot|>"

reply = None
refresh_args = {
		'max_length': 1000,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 40,
	}

title_regex = r"<\|sot\|>(.+?)<\|eot\|>"
text_body_regex = r"<\|sost\|>(.+?)<\|eost\|>"

while reply is None:
    for text in text_model_generator.generate(prompt=prompt, args=refresh_args, verbose=True):
        foo = text.replace(prompt, "<|sot|>")
        result = foo.split("<|eost|>")[0] + "<|eost|>"
        title = re.findall(title_regex, result)[0]
        body = re.findall(text_body_regex, result)[0]
        result = title + "\n" + body
        if result != None:
            reply = result
            break
print(reply)

